In [1]:
#from google.colab import files
#uploaded = files.upload()

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
churnData = pd.read_csv('/Users/louis/Desktop/customer_churn.csv')
churnData.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
#type(churnData['TotalCharges'].loc[0])
type(float("   "))

ValueError: ignored

In [9]:
display(churnData['TotalCharges'].value_counts())
rows_to_drop = list(churnData[churnData['TotalCharges'] == ' '].index)
churnData.drop(churnData.index[rows_to_drop],inplace=True)
display(churnData['TotalCharges'].value_counts())
churnData['TotalCharges'] = list(map(float,churnData['TotalCharges']))

20.2       11
           11
19.75       9
20.05       8
19.65       8
           ..
3121.1      1
1815.65     1
44.95       1
1078.9      1
6474.45     1
Name: TotalCharges, Length: 6531, dtype: int64

20.2       11
19.75       9
19.65       8
19.9        8
20.05       8
           ..
3121.1      1
1815.65     1
44.95       1
1078.9      1
6474.45     1
Name: TotalCharges, Length: 6530, dtype: int64

In [10]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [11]:
churnData['Churn'].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

As you can see there is a huge imbalance in the representation of the two categories 

In [12]:
5163/(5163+1869)

0.7342150170648464

In [13]:
churnData.drop(columns="customerID",inplace=True)

In [14]:
cat_cols = churnData.select_dtypes(include='object')
cat_cols.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,Female,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,No
1,Male,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,No
2,Male,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,Yes
3,Male,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),No
4,Female,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,Yes


One hot enconding

In [15]:
churnData['gender'] = np.where(churnData['gender'] == 'Male',1,0)
my_cols = ['Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection',
           'TechSupport','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']

for col in my_cols:
  churnData[col] = np.where(churnData[col] == 'Yes',1,0)

churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,No
1,1,0,0,0,34,1,No,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,No
2,1,0,0,0,2,1,No,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,Yes
3,1,0,0,0,45,0,No phone service,DSL,1,0,1,0,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,No
4,0,0,0,0,2,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,Yes


In [16]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
cat.head()

,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0,1,0


In [17]:
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,1,0,0,0,0,1,29.85,29.85,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,1,0,0,0,34,1,1,0,1,0,0,0,0,56.95,1889.50,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,2,1,1,1,0,0,0,0,1,53.85,108.15,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1,0,0,0,45,0,1,0,1,0,0,0,0,42.30,1840.75,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,2,1,0,0,0,0,0,0,1,70.70,151.65,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [18]:
y = pd.DataFrame(data=churnData, columns=['Churn'])
transformer = StandardScaler().fit(X)
scaled_x = transformer.transform(X)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=100)

In [20]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model is: %4.2f "% (classification.score(X_test, y_test)))

The accuracy of the logistic_regression model is: 0.80 


With this imbalance, when we tried to fit the model it still gave us an accuracy of 80%

In [21]:
y.value_counts()

Churn
No       5163
Yes      1869
dtype: int64

Note: Even if we just blindly say that each prediction is **No**, we would still get an accuracy of 

In [22]:
print("The accuracy of a blind guess is: %4.2f " % (5163/(5163+1869)))

The accuracy of a blind guess is: 0.73 


In [23]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

The kappa of the logistic regression model is: 0.47 


# Lets increase the imbalance and see how the model works 

In [24]:
data = pd.concat([X,y], axis=1, sort=False)
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,0,0,1,0,1,0,0,1,0,0,0,0,1,29.85,29.85,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,No
1,1,0,0,0,34,1,1,0,1,0,0,0,0,56.95,1889.50,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,No
2,1,0,0,0,2,1,1,1,0,0,0,0,1,53.85,108.15,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,Yes
3,1,0,0,0,45,0,1,0,1,0,0,0,0,42.30,1840.75,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,No
4,0,0,0,0,2,1,0,0,0,0,0,0,1,70.70,151.65,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,Yes


In [25]:
yes = data[data['Churn']=='Yes']
no = data[data['Churn']=='No']
yes = yes.sample(500)

In [26]:
data = pd.concat([yes,no], axis=0)
print(data['Churn'].value_counts())
data.head()

No     5163
Yes     500
Name: Churn, dtype: int64


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
2924,1,0,0,0,11,1,0,1,0,0,0,0,1,74.55,824.75,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,Yes
5967,1,1,0,0,4,0,0,0,0,0,0,0,1,25.20,102.50,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,Yes
4149,1,0,1,0,71,0,0,1,0,0,1,1,0,49.35,3515.25,1,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,1,0,1,0,Yes
3423,1,0,0,1,1,1,1,0,0,0,0,0,0,50.60,50.60,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Yes
1176,0,0,0,0,6,1,0,1,1,0,1,1,1,109.90,669.45,0,0,0,1,0,1,1,0,1,1,1,0,1,1,0,0,0,1,0,0,Yes


In [27]:
#shuffling the data
data = data.sample(frac=1)
data['Churn'].value_counts()

No     5163
Yes     500
Name: Churn, dtype: int64

In [28]:
X = data.drop('Churn', axis=1)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
6130,0,0,0,0,58,1,1,1,1,0,0,0,1,64.90,3795.45,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,1,1,0,0
2272,1,0,0,1,1,1,0,0,1,0,0,0,1,49.85,49.85,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
5997,0,0,0,0,6,1,0,1,0,0,1,0,1,89.75,552.65,0,0,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,0
11,1,0,0,0,16,1,0,0,0,0,0,0,0,18.95,326.80,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0
4705,1,0,0,0,6,1,1,1,0,0,0,0,1,78.95,494.95,1,0,0,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [29]:
y = data['Churn']
transformer = StandardScaler().fit(X)
scaled_x = transformer.transform(X)

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=100)

In [31]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model is: %4.2f "% (classification.score(X_test, y_test)))

The accuracy of the logistic_regression model is: 0.92 


In [32]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

The kappa of the logistic regression model is: 0.08 


In [34]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, labels=['Yes','No']))

              precision    recall  f1-score   support

         Yes       0.88      0.05      0.09       143
          No       0.92      1.00      0.96      1556

    accuracy                           0.92      1699
   macro avg       0.90      0.52      0.53      1699
weighted avg       0.92      0.92      0.88      1699



Despite the advantage of balancing classes, these techniques also have their weaknesses (there is no free lunch). The simplest implementation of over-sampling is to duplicate random records from the minority class, which can cause overfitting. In under-sampling, the simplest technique involves removing random records from the majority class, which can cause loss of information.

In [35]:
counts = churnData['Churn'].value_counts()
yes = churnData[churnData['Churn']=='Yes'].sample(counts[0], replace=True)
no = churnData[churnData['Churn']=='No']
data = pd.concat([yes,no], axis=0)
data = data.sample(frac=1)
data['Churn'].value_counts()

No     5163
Yes    5163
Name: Churn, dtype: int64

In contrast, before we had.

In [36]:
counts = churnData['Churn'].value_counts()
counts

No     5163
Yes    1869
Name: Churn, dtype: int64

In [37]:
cat = pd.get_dummies(data[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = data[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = data.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
4073,1,0,1,1,14,1,1,1,0,0,1,0,1,65.45,937.60,1,1,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0
2798,1,0,0,0,10,1,0,0,1,0,1,1,1,94.90,1048.85,1,0,0,1,0,0,1,0,1,1,1,0,0,1,0,0,0,0,1,0
5145,1,0,1,1,66,1,0,0,0,0,0,0,1,25.70,1714.55,1,1,1,1,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,0
4323,0,0,0,0,32,1,0,0,0,0,1,0,0,84.05,2781.85,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0
3185,0,1,0,0,1,0,0,0,0,0,0,0,0,25.05,25.05,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [38]:
transformer = StandardScaler().fit(X)
scaled_x = transformer.transform(X)
y = data['Churn']

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=100)

In [40]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model is: %4.2f "% (classification.score(X_test, y_test)))

The accuracy of the logistic_regression model is: 0.76 


In [41]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

The kappa of the logistic regression model is: 0.52 


In [42]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, labels=['Yes','No']))

              precision    recall  f1-score   support

         Yes       0.74      0.80      0.77      1547
          No       0.79      0.72      0.75      1551

    accuracy                           0.76      3098
   macro avg       0.76      0.76      0.76      3098
weighted avg       0.76      0.76      0.76      3098



F1 = 2 * (Precission*Recall)/(Pprecission+Recall)

In [43]:
2* ((0.74*0.80)/(0.74+0.80))

0.7688311688311688

Other more sophisticated resampling techniques include:

Cluster the records of the majority class, and do the under-sampling by removing records from each cluster, thus seeking to preserve information. 
In over-sampling, instead of creating exact copies of the minority class records, we can introduce small variations into those copies, creating more diverse synthetic samples.

In [35]:
#pip install imblearn

In [44]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,1,0,0,0,0,1,29.85,29.85,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,1,0,0,0,34,1,1,0,1,0,0,0,0,56.95,1889.50,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,2,1,1,1,0,0,0,0,1,53.85,108.15,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1,0,0,0,45,0,1,0,1,0,0,0,0,42.30,1840.75,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,2,1,0,0,0,0,0,0,1,70.70,151.65,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [45]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Downsampling
rus = RandomUnderSampler() 
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_rus, y_rus = rus.fit_sample(X, y)

In [46]:
y.value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

In [47]:
y_rus = pd.DataFrame(data=y_rus.flatten())
y_rus.value_counts()

Yes    1869
No     1869
dtype: int64

In [48]:
transformer = StandardScaler().fit(X_rus)
X = transformer.transform(X_rus)

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_rus, test_size=0.3, random_state=100)

In [50]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model after undersampling is: %4.2f "% (classification.score(X_test, y_test)))

The accuracy of the logistic_regression model after undersampling is: 0.76 


In [51]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model after undersampling is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

The kappa of the logistic regression model after undersampling is: 0.52 


In [52]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, labels=['Yes','No']))

              precision    recall  f1-score   support

         Yes       0.73      0.80      0.76       547
          No       0.79      0.72      0.75       575

    accuracy                           0.76      1122
   macro avg       0.76      0.76      0.76      1122
weighted avg       0.76      0.76      0.76      1122



In [53]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,1,0,0,0,0,1,29.85,29.85,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,1,0,0,0,34,1,1,0,1,0,0,0,0,56.95,1889.50,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,2,1,1,1,0,0,0,0,1,53.85,108.15,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1,0,0,0,45,0,1,0,1,0,0,0,0,42.30,1840.75,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,2,1,0,0,0,0,0,0,1,70.70,151.65,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [54]:
ros = RandomOverSampler()
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_ros, y_ros = ros.fit_sample(X, y)

In [55]:
y.value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

In [56]:
y_ros = pd.DataFrame(data=y_ros.flatten())
y_ros.value_counts()

Yes    5163
No     5163
dtype: int64

In [57]:
transformer = StandardScaler().fit(X_ros)
X = transformer.transform(X_ros)

In [58]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model after oversampling is: %4.2f "% (classification.score(X_test, y_test)))

The accuracy of the logistic_regression model after oversampling is: 0.76 


In [59]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model after undersampling is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

The kappa of the logistic regression model after undersampling is: 0.52 


In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, labels=['Yes','No']))

              precision    recall  f1-score   support

         Yes       0.73      0.80      0.76       547
          No       0.79      0.72      0.75       575

    accuracy                           0.76      1122
   macro avg       0.76      0.76      0.76      1122
weighted avg       0.76      0.76      0.76      1122



### Synthetic Minority Oversampling TEchnique (SMOTE)

In [61]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,1,0,0,0,0,1,29.85,29.85,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,1,0,0,0,34,1,1,0,1,0,0,0,0,56.95,1889.50,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,2,1,1,1,0,0,0,0,1,53.85,108.15,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1,0,0,0,45,0,1,0,1,0,0,0,0,42.30,1840.75,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,2,1,0,0,0,0,0,0,1,70.70,151.65,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [62]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_sm, y_sm = smote.fit_sample(X, y)
y_sm = pd.DataFrame(data=y_sm.flatten())
y_sm.value_counts()

Yes    5163
No     5163
dtype: int64

### UnderSampling using TomekLinks 

Tomek links are pairs of very close instances, but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [53]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_sample(X, y)
y_tl = pd.DataFrame(data=y_tl.flatten())
y_tl.value_counts()

No     4615
Yes    1869
dtype: int64

In [54]:
X_tl2, y_tl2 = tl.fit_sample(X_tl, y_tl)
y_tl2 = pd.DataFrame(data=y_tl2.flatten())
y_tl2.value_counts()

No     4460
Yes    1869
dtype: int64

In [55]:
# It does not make the two classes equal but only removes the points from the majority 
# class that are close to other poitns in minority class

In [56]:
# Similar way it works for multi label classification models 
# But it can be a little tricky 

In [57]:
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']

In [58]:
y[6000:7000] = 'Hello'
y.value_counts()

No       4438
Yes      1594
Hello    1000
Name: Churn, dtype: int64

In [59]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_sample(X, y)
y_tl = pd.DataFrame(data=y_tl.flatten())
y_tl.value_counts()

No       3704
Yes      1594
Hello    1000
dtype: int64

In [60]:
X_sm, y_sm = smote.fit_sample(X, y)
y_sm = pd.DataFrame(data=y_sm.flatten())
y_sm.value_counts()

Yes      4438
No       4438
Hello    4438
dtype: int64